<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/040_GAME_Agent_Design_Pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let’s walk through each structural difference **one at a time**, with clear explanations and examples from both the **Modular Agent** and the **Manual Agent**.

---

# 1. ✅ **Design Pattern**

| Modular Agent (New)                                                          | Manual Agent (Old)                                        |
| ---------------------------------------------------------------------------- | --------------------------------------------------------- |
| Uses the **GAME** framework: `Goals`, `Actions`, `Memory`, and `Environment` | Everything lives in a flat script with hardcoded behavior |

#### 🔍 Explanation:

* **Modular Agent** has clearly separated components:

  * `Goal`: What is the agent trying to achieve?
  * `Action`: Registered tools with formal schemas
  * `Memory`: All past responses and tool results
  * `Environment`: Executes tool logic in isolation

* **Manual Agent**:

  * You have `tool_router()` manually matching names
  * All logic happens in one loop; no separation of concerns

#### ✅ Why it matters:

Structured design helps you debug, extend, and test parts individually. For example, you can swap out the environment for a GitHub Actions-based runner without touching the agent logic.

---

Starting simple is the best way to make the differences crystal clear. We'll focus on a **basic tool** (e.g., listing files in a directory), and compare:

1. ✅ The **Manual Agent** (flat, hardcoded logic)
2. 🎯 The **Modular GAME Agent** (Goal, Action, Memory, Environment separated)

---

## 🧪 Manual Agent (Old Approach) – Example 1: List Files

Here’s a simple manual implementation:

```python
import os

def list_files():
    return os.listdir(".")

def tool_router(tool_name):
    if tool_name == "list_files":
        return list_files()
    else:
        return "Unknown tool"

def run_manual_agent(user_input):
    if "list" in user_input:
        result = tool_router("list_files")
        print("Files:", result)
    else:
        print("Sorry, I can’t help with that.")

# Simulate usage
run_manual_agent("Can you list the files?")
```

### 🔍 Key Traits:

* No goals — just hardcoded keyword detection
* No separation between logic, action, and environment
* No memory
* Any changes require editing the `run_manual_agent()` and `tool_router()` functions

---



## ✅ Modular Agent (GAME Framework) – Example 1: List Files

Now here’s the **same feature** implemented using the **GAME** structure, just simplified to focus on design.

### 1. Define the Goal:

```python
from dataclasses import dataclass

@dataclass(frozen=True)
class Goal:
    name: str
    description: str
```

```python
goals = [Goal(name="Explore Files", description="List files in the current directory")]
```

---

### 2. Register the Action:

```python
from typing import List, Callable

class Action:
    def __init__(self, name: str, function: Callable, description: str, terminal: bool = False):
        self.name = name
        self.function = function
        self.description = description
        self.terminal = terminal

class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action: Action):
        self.actions[action.name] = action

    def get_action(self, name: str):
        return self.actions.get(name)
```

```python
import os

def list_files():
    return os.listdir(".")

action_registry = ActionRegistry()
action_registry.register(Action(
    name="list_files",
    function=list_files,
    description="List files in the current directory."
))
```

---

### 3. Define the Agent:

```python
class SimpleAgent:
    def __init__(self, goals, action_registry):
        self.goals = goals
        self.action_registry = action_registry

    def run(self, user_input):
        if "list" in user_input:
            action = self.action_registry.get_action("list_files")
            result = action.function()
            print("Files:", result)
        else:
            print("I’m not sure what to do.")

# Run it
agent = SimpleAgent(goals, action_registry)
agent.run("Can you list the files?")
```

---

## ✅ Why this is Better:

* You could easily **swap in a new environment** later, e.g., for GitHub instead of local OS
* If you want to add more tools, just `register()` more actions
* Goals are tracked (and could be used in prompt construction)
* Much easier to test the components in isolation (goal, action, logic)

---

## 🧪 Old Approach (Manual Agent)

### 🧱 Architecture:

* You write **independent utility functions** (`list_files`, `read_file`, etc.)
* Then you create a **manual control loop** or `tool_router()` to call them
* You **manually decide** which function to run based on keywords or logic

### 🤖 How the LLM interacts:

* The LLM isn't really "in charge"
* It gives plain text input, and you write logic to figure out which function to call
* **No memory, no autonomy, no higher-level intent**





## ✅ New Approach (GAME Agent)

### 🧱 Architecture:

* You define a **Goal** (or multiple) — e.g., *"Explore files in a project"*
* You register **Actions** (tools) with descriptions and parameter schemas
* You define a **Memory** and an **Environment**
* The LLM is orchestrated by an **Agent Loop** to:

  1. See its goal(s)
  2. See available tools
  3. Consider memory (context)
  4. Select an action
  5. Execute the action and reflect

### 🤖 How the LLM interacts:

* The LLM sees a structured **prompt** with:

  * The *goal* it's trying to achieve
  * The *tools* it can use (with descriptions and schemas)
  * The *memory* (what has happened so far)
  * The *environment* constraints
* It chooses an action using **function calling**, and gets structured results
* The agent decides when to **terminate**, ask the user for help, or continue

---

## 🔍 Key Insight:

> In the old model, **you were the agent**.
> In the new model, **you build an agent**, and the LLM becomes the brain inside it.

That’s the power of the GAME architecture — you’re shifting from building "helpers" to building actual **thinking agents** that pursue objectives with the resources you give them.





The **tool function implementations** themselves (like `list_files`, `read_file`, and `terminate`) stay exactly the same in both the old and new approaches.

---

### 🔧 Why the tool definitions don’t change:

The actual logic of each function — what it *does* — belongs in your **environment** (i.e., your Python runtime). That’s true in both models:

| Part           | Old Agent                           | New Agent (GAME)                            |
| -------------- | ----------------------------------- | ------------------------------------------- |
| Tool function  | Defined as regular Python code      | Defined as regular Python code              |
| Tool access    | Manually matched in `tool_router()` | Registered in the `ActionRegistry`          |
| Tool execution | Called directly                     | Executed via `environment.execute_action()` |

---

### 🧠 What **does** change?

What *does* change is:

1. **How tools are exposed** to the agent:

   * In the old way: You use manual routing logic
   * In GAME: You use `Action(...)` objects with name, description, parameters, and terminal status

2. **How the agent chooses and uses tools**:

   * In the old way: You decide which tool to call
   * In GAME: The **LLM decides** which tool to call based on its goal

---

### ✅ So you're on the right path:

You **only write tool functions once** — and then make them "agent-ready" by registering them with an `ActionRegistry`. That’s it.


Mastering `Action` and `ActionRegistry` is a big step toward becoming fluent with the **GAME agent architecture**. Here's a breakdown of the **key components and concepts you should learn**:

---

## 🧱 1. `Action` Class – What Is an Action?

The `Action` class wraps a **tool** (your function) with metadata to make it usable inside the agent framework.

### ✅ Key Attributes

| Attribute     | Purpose                                                            |
| ------------- | ------------------------------------------------------------------ |
| `name`        | Unique identifier for the tool. Must match what the LLM will use.  |
| `function`    | The actual Python function to call (e.g., `read_file`).            |
| `description` | Human-readable explanation. Can be used in prompt construction.    |
| `terminal`    | A flag indicating whether using this tool **ends the agent loop**. |

### 🧠 Why it's useful

Instead of having tools float around in different places, `Action` gives the agent a structured way to understand:

* What tools are available
* What they do
* Whether using one should **end the task** (like `terminate()`)

---

## 🔧 What Is the `Action` Class?

The `Action` class is a **structured wrapper** around a tool (function), turning it into a **well-defined, controllable unit** the agent can reason about and interact with.

### Instead of just:

```python
def read_file(file_name): ...
```

We now wrap that in:

```python
Action(
    name="read_file",
    function=read_file,
    description="Reads the content of a file in the current directory.",
    terminal=False
)
```

This turns a raw function into something the **agent can select, describe, reason over, and execute safely.**

---

## ✅ Why Is This Valuable?

### 1. **Provides Metadata**

The `Action` class gives more than just a callable function — it supplies:

| Field         | Purpose                                                                 |
| ------------- | ----------------------------------------------------------------------- |
| `name`        | What the agent refers to this action as (like an OpenAPI `operationId`) |
| `function`    | The actual Python code to run when this tool is used                    |
| `description` | Natural language hint for the LLM or user interface                     |
| `terminal`    | Special flag to tell the agent: “If you run this, you’re done”          |

Without `Action`, you'd be **scattering this info manually** in your routing logic or prompt generation.

---

### 2. **Encapsulation**

Encapsulating all info about an action (tool) in one object means:

* It can be **registered once**, and
* Used **repeatedly and predictably**

No more rewriting logic like:

```python
if tool_name == "read_file":
    call read_file()
```

Instead:

```python
action = action_registry.get_action("read_file")
action.function(**args)
```

---

### 3. **Enables Scalable Tooling**

As your agent grows:

* With many tools
* With goals that span tools
* With custom interfaces (e.g., GUI, CLI, notebook, API)

…the `Action` class provides a **clean contract** for how those tools are described and used.

---

## 🆚 How It Differs from Early Tool Setup

| Concept                 | Early Tool Setup                          | Action Class Approach                         |
| ----------------------- | ----------------------------------------- | --------------------------------------------- |
| **Function definition** | Raw Python functions                      | Python functions wrapped in metadata          |
| **Routing**             | Manual tool name matching (e.g., if-else) | Dynamic lookup via name → object mapping      |
| **Prompt construction** | Must manually inject tool descriptions    | Pulls from `action.description` automatically |
| **Terminal tools**      | Must hardcode logic                       | Handled via `terminal=True` flag              |
| **Scalability**         | Gets messy fast                           | Clean, extensible, declarative                |

---

## 📦 Analogy: Think of `Action` as an API Endpoint Object

Imagine building a REST API with just Python functions. It would work, but you’d be missing:

* A name for each endpoint
* A description
* Whether it’s GET or POST
* Parameter requirements
* Whether it ends a workflow

That’s what `Action` gives you — **structured metadata around your raw functions** so the agent (and you!) can manage them in a clean, powerful way.





## 📦 2. `ActionRegistry` – Centralized Tool Management

The `ActionRegistry` acts like a **toolbox**: a place where all actions are registered and retrievable by name.

### ✅ Key Methods

| Method                       | Purpose                                             |
| ---------------------------- | --------------------------------------------------- |
| `register(action)`           | Adds an `Action` to the registry.                   |
| `get_action(name)`           | Fetches an `Action` by its name.                    |
| `get_actions()` *(optional)* | Returns all actions – useful when building prompts. |

You only register each tool once. After that, the agent can dynamically look up and invoke the tool based on LLM output.

---

## 🎯 Core Learning Goals

Here's what you should take away:

| Concept                | Why It Matters                                                                                                           |
| ---------------------- | ------------------------------------------------------------------------------------------------------------------------ |
| **Naming Consistency** | The name you register in `Action(name=...)` must match the name used in LLM responses.                                   |
| **Decoupling Logic**   | Tools are no longer hardcoded in if/else routers — they are abstracted and managed cleanly.                              |
| **Extendability**      | Want to add a new tool? Just define the function, wrap it in an `Action`, and register it. No other code changes needed. |
| **Terminal Actions**   | Use `.terminal=True` to let the agent know when it's time to end (like with `terminate`).                                |

---

## 🛠️ Example Summary

```python
# Your tool function
def list_files():
    return os.listdir(".")

# Wrap in an Action
action = Action(
    name="list_files",
    function=list_files,
    description="Lists files in the current directory.",
    terminal=False
)

# Register the action
action_registry = ActionRegistry()
action_registry.register(action)

# Later, your agent does:
action = action_registry.get_action("list_files")
result = action.function()  # calls the tool!
```






### ✅ What Is `ActionRegistry`?

`ActionRegistry` is a **container for actions**. Think of it like a toolbox:

* Each `Action` is a tool.
* The `ActionRegistry` is the organized toolbox that:

  * Keeps track of all tools (by name)
  * Lets the agent **look up** the right tool by name
  * Ensures the agent only chooses from **registered** (known, safe) tools

---

## 🛠️ Why Use It?

### 1. **Ease of Use**

You can register actions in one line:

```python
action_registry.register(Action(...))
```

You don’t need to manually track what tools the agent has access to — it’s all encapsulated in the registry.

---

### 2. **Scalability**

You can register **as many actions as you want**, from different modules, categories, or files:

```python
action_registry.register(load_data_action)
action_registry.register(run_query_action)
action_registry.register(send_email_action)
```

If your agent grows from 3 tools to 30 — no problem.

---

### 3. **Dynamic Access**

The agent doesn’t need to hardcode which function to use. It just says:

```python
action = action_registry.get_action("read_file")
result = action.function(**args)
```

So even if you later change or re-implement the function behind `"read_file"`, the agent loop stays untouched.

---

### 4. **Encapsulation & Safety**

Since the registry contains **only allowed actions**, the agent can’t call unregistered or dangerous functions. It’s a **controlled interface** between LLM and Python.

---

## 📦 Analogy: Plugin System

Think of `ActionRegistry` like a **plugin manager**:

* Actions are plugins.
* You register each plugin.
* The core engine (agent) doesn’t care what each plugin does — it just knows how to call it.

This is **classic object-oriented design** — flexible, extensible, and decoupled.





### 2. ✅ **Extensibility**

| Modular Agent (New)                                   | Manual Agent (Old)                                                    |
| ----------------------------------------------------- | --------------------------------------------------------------------- |
| Add a new action with `action_registry.register(...)` | Must update `tool_router`, schema list, and response parsing manually |

#### 🔍 Explanation:

* In the modular setup, adding a new tool is as easy as:

  ```python
  action_registry.register(Action(
      name="summarize_file",
      function=summarize_file,
      ...
  ))
  ```

* In the manual agent:

  * You'd define the tool function
  * Manually add a `tool_schema` entry
  * Update `tool_router()` to handle it
  * Adjust parsing logic to route and return results

#### ✅ Why it matters:

This saves time and reduces human error, especially as your agent grows from 3 tools to 30+.

---

### 3. ✅ **Memory**

| Modular Agent (New)                        | Manual Agent (Old)                                       |
| ------------------------------------------ | -------------------------------------------------------- |
| `Memory` is a class that tracks every step | Only the user input or last result is stored (if at all) |

#### 🔍 Explanation:

* Modular Agent:

  ```python
  memory.add_memory({"type": "user", "content": user_input})
  memory.add_memory({"type": "assistant", "content": response})
  ```

* Manual Agent:

  ```python
  messages = [{"role": "user", "content": user_input}]
  # No long-term memory between calls
  ```

#### ✅ Why it matters:

Memory enables the agent to **reason over time**, avoid repeating itself, and build on prior results. This becomes critical for multi-step tasks.

---

### 4. ✅ **Environment Abstraction**

| Modular Agent (New)                                             | Manual Agent (Old)                    |
| --------------------------------------------------------------- | ------------------------------------- |
| Uses an `Environment()` class with an `execute_action()` method | Functions are called directly by name |

#### 🔍 Explanation:

* Modular Agent:

  ```python
  result = environment.execute_action(action, invocation["args"])
  ```

* Manual Agent:

  ```python
  result = tool_router(tool_name, args)
  ```

#### ✅ Why it matters:

The environment can later:

* Log action execution
* Simulate execution in tests
* Run in cloud / serverless contexts

And you don’t have to change the agent logic to support those.

---

### 5. ✅ **Function Calling**

| Modular Agent (New)                       | Manual Agent (Old)                             |
| ----------------------------------------- | ---------------------------------------------- |
| Uses OpenAI’s structured function calling | Manually parsed JSON or tool\_calls if present |

#### 🔍 Explanation:

* Modular Agent:

  ```python
  agent_language = AgentFunctionCallingActionLanguage()
  # All parsing and schema validation is handled internally
  ```

* Manual Agent:

  ```python
  if response.choices[0].message.tool_calls:
      tool_name = ...
      args = json.loads(...)
  ```

#### ✅ Why it matters:

Letting OpenAI handle schema validation and argument generation means:

* Fewer errors
* Less code
* No manual JSON parsing


